In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
#nltk.download()


In [33]:
Data = pd.read_csv("Train_E&I_L0.csv", sep='|' , delimiter=',' ,encoding='latin-1')

In [34]:
col = ['Material/Service Description','I06: Material Group','L0','L1','L2']
Train_data = Data[col]
Train_data.rename(columns = {'Material/Service Description' : 'Desc', 'I06: Material Group' : 'MG'}, inplace = True)
Train_data.head()

In [41]:
TrainData = Train_data.loc[Train_data.L1 != 'NON']

In [43]:
TrainData["L1"].value_counts()
#TrainData["L2"].value_counts()

Electrical         2072
Instrumentation    1734
E&I Bulk           1524
Name: L1, dtype: int64

In [44]:
TrainData['Desc'] = TrainData['Desc'].astype(str)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [45]:
import re
def clean_str(string):
    string = re.sub(r"\n","", string)
    string = re.sub(r"\r", "", string)
    #string = re.sub(r"[0-9]","digit", string)
    string = re.sub(r"\'", " ", string)
    string = re.sub(r"\"", " ", string)
    string = re.sub(r'\(.*?\)', '', string)
    string = re.sub(r"/", " ", string)
    string = re.sub(r":", " ", string)
    string = re.sub(r"-", " ", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"\+", " ", string)
    string = re.sub(r"&"," ", string)
    string = re.sub(r"#"," ", string)
    string = re.sub(r"\."," ", string)
    string = re.sub(r"\b\w+\d+\b"," ", string)
    string = re.sub(r"\b\d+\w+\b"," ", string)
    string = re.sub(r"\b\w+\d+\w+\b"," ", string)
    string = re.sub(r"[0-9]","", string)
    string = re.sub(r"\\b[A-Za-z] \\b|\\b [A-Za-z]\\b", ' ', string)
    string = re.sub(r'\b\w\b', '', string)
    string = re.sub(r'\b\w\w\b', '', string)
    string = re.sub(r'" "', '', string)
    string = re.sub(' +', ' ', string)
    string = re.sub(r"[^\w]"," ", string)
    #string = re.sub('cons\s', "consumables " " ", str)
    return string.strip().lower()


In [47]:
def replace_str(string):
    string = re.sub("cons\s", "consumables " " ", string)
    string = re.sub("chem\s", "chemicals " " ", string)
    string = re.sub("geotech\s", "geotechnical" " ", string)
    string = re.sub("test\s", "testing " " ", string)
    string = re.sub("inst\s", "instrument " " ", string)
    string = re.sub("transport\s", "transportation " " ", string)
    string = re.sub("rig\s", "rigging " " ", string)
    string = re.sub("lift\s", "lifting " " ", string)
    string = re.sub("cables\s", "cable  " " ",     string)
    string = re.sub("fittin\s", "fittings " " ", string)
    string = re.sub("peri\s", "peripherals " " ", string)
    string = re.sub(r'" "', '', string)
   
    return string.strip().lower()

In [46]:
TrainData['Feature'] = TrainData['Desc'].apply(lambda x: clean_str(x))

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
TrainData['Feature'] = TrainData['Feature'].apply(replace_str)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
TrainData.head()

,Desc,MG,L0,L1,L2,Feature
0,"BULB, NAV PANEL, 24V, COLLER CODE: BLUE",MEL010000,Electrical & Instrumentation,Electrical,Electrical Control Panels and Control Stations,bulb nav panel coller code blue
1,"BULB, NAV PANEL, 24V, COLLER CODE: GREE",MEL010000,Electrical & Instrumentation,Electrical,Electrical Control Panels and Control Stations,bulb nav panel coller code gree
2,"BULB, NAV PANEL, 24V, COLLER CODE: RED",MEL010000,Electrical & Instrumentation,Electrical,Electrical Control Panels and Control Stations,bulb nav panel coller code red
3,"BULB, NAV PANEL, 24V, COLLER CODE: YELL",MEL010000,Electrical & Instrumentation,Electrical,Electrical Control Panels and Control Stations,bulb nav panel coller code yell
4,"MARINE PANEL , MPCF1-10.4"", (D), STBD",MEL010000,Electrical & Instrumentation,Electrical,Electrical Control Panels and Control Stations,marine panel stbd


In [50]:
#Clean = pd.DataFrame(TrainData).to_csv('Clean.csv')

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

#X_train, X_test, y_train, y_test = train_test_split(TrainData['Feature'], TrainData['L1'],test_size=0.0, random_state = 0)
X_train = TrainData['Feature']
y_train = TrainData['L1']
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [52]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train_tfidf, y_train)

In [20]:
svc = SVC(kernel="linear")
SVM = svc.fit(X_train_tfidf, y_train)

In [53]:
X_train_counts.shape

(5330, 1715)

In [22]:

#y_pred = clf.predict(X_test)

In [23]:
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [24]:
#from sklearn.metrics import classification_report, confusion_matrix
#print(confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred))

In [98]:

Test = pd.read_excel('C:/Users/user/Desktop/Classify/Results/Test_E&I_2016_L1.xlsx')

In [99]:
#Test_Filter = Test.loc[Test.L0 != 'NON']

In [100]:
col = ['Material/Service Description','L0']
Test_data = Test[col]
Test_data.rename(columns = {'Material/Service Description' : 'Desc'}, inplace = True)
Test_data.head()

In [103]:
Test_data['L0'].value_counts()

Electrical & Instrumentation    1135
Name: L0, dtype: int64

In [104]:
Test_data.head()

,Desc,L0
0,(DO NOT USE)INSTRUMENT-TUBING,Electrical & Instrumentation
1,(DO NOT USE)INSTRUMENT-FITTINGS,Electrical & Instrumentation
2,(DO NOT USE)INSTRUMENT-TUBING,Electrical & Instrumentation
3,(DO NOT USE)INSTRUMENT-TUBING,Electrical & Instrumentation
4,INSTRUMENT & UTILITY GAS SYSTEMS,Electrical & Instrumentation


In [105]:
Test_data['Feature'] = Test_data['Desc'].apply(lambda x: clean_str(x))
Test_data['Feature'] = Test_data['Feature'].apply(replace_str)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [106]:
Test_data['Feature'].head()

0                 instrument tubing
1               instrument fittings
2                 instrument tubing
3                 instrument tubing
4    instrument utility gas systems
Name: Feature, dtype: object

In [114]:
#Clean = pd.DataFrame(Test_data).to_csv('C:/Users/user/Desktop/Classify/Results/Clean.csv')

In [107]:
X_test = count_vect.transform(Test_data['Feature'])

In [108]:
y_pred = clf.predict(X_test)
Y_pred = SVM.predict(X_test)

In [109]:
Pred = pd.DataFrame(y_pred, columns = ['L1(Tree)'])
Result = pd.concat([Test, Pred], axis=1)
#prediction = pd.DataFrame(Result).to_excel("C:/Users/user/Desktop/Classify/Results/prediction2016Tree.xlsx")

In [113]:
Prediction = pd.DataFrame(Y_pred, columns = ['L1(SVM)'])
Results = pd.concat([Result, Prediction], axis=1)
predictions = pd.DataFrame(Results).to_excel("C:/Users/user/Desktop/Classify/Results/prediction2016L1.xlsx")